In [73]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import inception_v3
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dense,Activation,Flatten,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import numpy as np
import cv2,os
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D

In [74]:

data_path='/content/drive/My Drive/dataset (1)'
categories=os.listdir(data_path)
labels=[i for i in range(len(categories))]







{'with_mask': 0, 'without_mask': 1}
['with_mask', 'without_mask']
[0, 1]


In [75]:
img_size=100
data=[]
target=[]


for category in categories:
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path)
        
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)

        try:
            gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)           
            #Coverting the image into gray scale
            resized=cv2.resize(gray,(img_size,img_size))
            #resizing the gray scale into 50x50, since we need a fixed common size for all the images in the dataset
            data.append(resized)
            target.append(label_dict[category])
            #appending the image and the label(categorized) into the list (dataset)

        except Exception as e:
            print('Exception:',e)
            #if any exception rasied, the exception will be printed here. And pass to the next image

In [76]:
data=np.array(data)/255.0
data=np.reshape(data,(data.shape[0],img_size,img_size,1))
target=np.array(target)
new_target=np_utils.to_categorical(target)

In [87]:
lb = LabelBinarizer()
target = lb.fit_transform(target)
target = to_categorical(target)

In [88]:
np.save('data',data)
np.save('target',new_target)

In [89]:
data=np.load('data.npy')
target=np.load('target.npy')


In [90]:
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint

model=Sequential()

model.add(Conv2D(200,(3,3),input_shape=data.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The first CNN layer followed by Relu and MaxPooling layers

model.add(Conv2D(100,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The second convolution layer followed by Relu and MaxPooling layers

model.add(Flatten())
model.add(Dropout(0.5))
#Flatten layer to stack the output convolutions from second convolution layer
model.add(Dense(50,activation='relu'))
#Dense layer of 64 neurons
model.add(Dense(2,activation='softmax'))
#The Final layer with two outputs for two categories

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [91]:
train_data,test_data,train_target,test_target=train_test_split(data,target,test_size=0.1)

In [95]:
checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
history=model.fit(train_data,train_target,epochs=20,callbacks=[checkpoint],validation_split=0.2)

Epoch 1/20
34/34 [==============================] - 83s 2s/step - loss: 0.6048 - accuracy: 0.6719 - val_loss: 0.4600 - val_accuracy: 0.8015
Epoch 2/20
34/34 [==============================] - 82s 2s/step - loss: 0.4093 - accuracy: 0.8157 - val_loss: 0.3467 - val_accuracy: 0.8493
Epoch 3/20
34/34 [==============================] - 83s 2s/step - loss: 0.2896 - accuracy: 0.8848 - val_loss: 0.2827 - val_accuracy: 0.8971
Epoch 4/20
34/34 [==============================] - 81s 2s/step - loss: 0.2188 - accuracy: 0.9115 - val_loss: 0.3012 - val_accuracy: 0.8934
Epoch 5/20
34/34 [==============================] - 82s 2s/step - loss: 0.1772 - accuracy: 0.9346 - val_loss: 0.2491 - val_accuracy: 0.9154
Epoch 6/20
34/34 [==============================] - 82s 2s/step - loss: 0.1310 - accuracy: 0.9539 - val_loss: 0.2543 - val_accuracy: 0.9007
Epoch 7/20
34/34 [==============================] - 82s 2s/step - loss: 0.1479 - accuracy: 0.9447 - val_loss: 0.2307 - val_accuracy: 0.9081
Epoch 8/20
34/34 [==

In [100]:
predIdxs = model.predict(test_data, batch_size=32)
predIdxs = np.argmax(predIdxs, axis=1)
print(classification_report(test_target.argmax(axis=1), predIdxs, target_names=['With Mask', 'Without Mask']))

              precision    recall  f1-score   support

   With Mask       0.99      0.96      0.97        72
Without Mask       0.96      0.99      0.97        79

    accuracy                           0.97       151
   macro avg       0.97      0.97      0.97       151
weighted avg       0.97      0.97      0.97       151

